# Command interface

DSC2 implements DSC by executing DSC script via command line program `dsc`:
## Help message

In [1]:
! dsc -h

usage: dsc [-h] [--version] [-v {0,1,2,3,4}] [-s str [str ...]] [-o str] [-f]
           [-r] [-j N] [--host str] [--remove str [str ...]]
           [--annotation str] [--annotate_to str] [--extract str]
           [--extract_from str] [--extract_to str] [--tags str [str ...]]
           DSC file

Implementation of Dynamic Statistical Comparisons

positional arguments:
  DSC file

optional arguments:
  -h, --help            show this help message and exit
  --version             show program's version number and exit
  -v {0,1,2,3,4}, --verbosity {0,1,2,3,4}
                        Output error (0), warning (1), info (2), debug (3) and
                        trace (4) information.
  -s str [str ...], --sequence str [str ...]
                        DSC sequence to be executed. It will override the
                        DSC::run entry when specified. Multiple sequences are
                        allowed. Each input should be a quoted string defining
                        a valid 

## DSC Execution options
* `-s` specifies which particular DSC sequence to run. See this [section](https://github.com/stephenslab/dsc2/wiki/04-Multiple-DSC-Sequences#execute-sequences-from-command-line) for an example.
*  `-j` specifies number of CPU threads to use. You may run DSC in parallel on a single desktop, or on distributed computer system that runs `Redis` server / worker (FIXME: link to tutorial), in which case `--host` will have to be specified.
*  `-o` changes the default output filename / directory specified in the DSC script.
*  `-f` forces executing DSC without using existing cache and runtime signature. During a DSC execution, cache files of intermediate output and runtime signatures of parameters are saved so that future re-runs of the same computational routine will be skipped. The `-f` option forces DSC to ignore existing cache and run the entire procedure afresh.
*  `-r` executes DSC using only existing cache but not runtime signature -- it will skip existing files by file name, without trying to figure out if the file property has been changed. This is useful when files are migrated from one computational system to another: `--recover` prevents DSC from re-running due to change of file properties in this case.
*  `-v` controls verbosity level. Default level is 2. If you encounter error messages you can increase the verbosity level to output more information to diagnose the cause.

## DSC results removal
*  `--remove` will remove some DSC steps as specified. In order to remove data properly, the dsc result database (defined in `dsc_file`) must be accessible.

## DSC result annotation
These options will annotate DSC by applying "tags" to DSC results. This is a pre-processing step to DSC results extraction

* `--annotation` specifies the annotation file. See this [section]() for an example annotation file.
* `--annotate_to` by default, is the name of the last DSC block if the DSC procedure only runs one sequence. Otherwise the name of the last DSC block has to be provided.

## DSC results extraction
These options will extract annotated DSC to a separate file for further analysis.

* `--extract` specifies the name of variable to extract. The variable name should be avaiable in the last DSC block.
* `--extract_from` same as `--annotate_to` above. 
* `--extract_to` output file for extracted data.
* `--tags` Annotations to extract.
